<H1>Estimator </H1>
<a href="https://quantum.cloud.ibm.com/docs/en/api/qiskit/primitives#overview-of-estimatorv2"> Estimator V2 </a>
<hr> 
<H2>Modification history </H2><br> 
<table>
    <tr>
        <th> Modified </th>
        <th> By </th>
        <th> Reason </th>
    </tr>
    <tr> 
        <td> 08-Jul-25 </td>
        <td> CBL </td> 
        <td> Looking at using AER and Estimator </td>
    </tr>
    <tr> 
        <td> 23-Jul-25 </td>
        <td> CBL </td> 
        <td> Deep dive on pauli operators and theta </td>
    </tr>
</table>
<hr> 
<H2> References </H2> 
<a href="https://docs.quantum.ibm.com/guides/install-qiskit"> Install Qikit </a> <br> 
<a href="https://docs.quantum.ibm.com/guides/hello-world"> qiskit hello world </a> <br> 
<a href="https://qiskit-community.github.io/qiskit-dynamics/"> Qiskit dynamics</a> <br> 
<a href="https://pypi.org/project/qiskit/"> pypi qiskit page</a> <br> 
<a href="https://docs.quantum.ibm.com/api/qiskit/visualization"> Visualization</a> <br> 
<a href="https://pypi.org/project/qiskit-aer/"> qiskit aer pypi</a> <br>
<a href="https://quantum.cloud.ibm.com/docs/en/api/qiskit"> qiskit aer documentation </a>
<br> 
<a href="https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.RealAmplitudes">Real Amplitudes - kind of important details here.</a>
<hr>
<H2> Notes </H2>
No simulator called out.
<br> 
<hr>

In [7]:
from qiskit.primitives import StatevectorEstimator as Estimator
# Updated from example. 
from qiskit.circuit.library import real_amplitudes
from qiskit.quantum_info import SparsePauliOp
import numpy as np

<H1> Estimator Operation </H1>
What we are trying to do is use a Hamiltonian of Pauli operations and then change the phase. 
<br> 
$\bra{\Psi}H\ket{\Psi}$

In [3]:
# Create the 'wavefunction', in this case 2 qubits. We are telling the system 
# that we want real, as opposed to complex, amplitudes returned. 
# reps is the number of times this iterative circuit is repeated. 
# this is really a convience function combining many operations into a single call 
psi1 = real_amplitudes(num_qubits=2, reps=2)
# two reps requires and 2 qubits requires 6 total angles. (3 rotations, one on each qubit.)
psi2 = real_amplitudes(num_qubits=2, reps=3)
#
# A selection of possible Hamiltonians to use. Each is a list with an Amplitude or associated weight, not sure
# what the weight does yet. 
#
H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])
# 
# And a multitude of angles. 
#
theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
theta3 = [1, 2, 3, 4, 5, 6]
# Create the estimmator 
estimator = Estimator()

In [2]:
# 
# calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
job = estimator.run([(psi1, H1, [theta1])])
job_result = job.result() # It will block until the job finishes.
print(f"The primitive-job finished with result {job_result}")

The primitive-job finished with result PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(1,), dtype=float64>), stds=np.ndarray(<shape=(1,), dtype=float64>), shape=(1,)), metadata={'target_precision': 0.0, 'circuit_metadata': {}})], metadata={'version': 2})


In [11]:
dir(job_result[0].data)

['_FIELDS',
 '_FIELD_TYPES',
 '_RESTRICTED_NAMES',
 '_SHAPE',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__non_callable_proto_members__',
 '__parameters__',
 '__protocol_attrs__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_data',
 '_is_protocol',
 '_is_runtime_protocol',
 '_shape',
 'evs',
 'items',
 'keys',
 'ndim',
 'shape',
 'size',
 'stds',
 'values']

In [30]:
job_result[0].data.evs

array([1.55555728])

In [15]:
# lets try some other stuff. 
# psi is the combined result. when completing the dot product, not intermediate 
# densities. 
#
H2 = SparsePauliOp.from_list([("IZ", 1)])
H0 = SparsePauliOp.from_list([("II", 1)])
# 
# And a multitude of angles. 
#
# Layout is qubit 0, then qubit 1 etc and repeating for the number of reps+1 (unless supress final is sepecified.)
# the angles are around the y axis 
# see printout below. 
#
theta0 = [np.pi/2, 0, np.pi/2, 0, 0, 0]
# 
# calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
# do we need to specify theta?, apparently yes
# why 6 operations??
#
job = estimator.run([(psi1, H0, [theta0])])
job_result = job.result() # It will block until the job finishes.
print("Result: ", job_result[0].data.evs)

Result:  [1.]


In [12]:
print(psi1)

     ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [16]:
print(H0)

SparsePauliOp(['II'],
              coeffs=[1.+0.j])


In [30]:
# lets make an even simplier circuit. 
#
psi0 = real_amplitudes(num_qubits=2, reps=1, skip_final_rotation_layer=True)
print(psi0)

     ┌──────────┐     
q_0: ┤ Ry(θ[0]) ├──■──
     ├──────────┤┌─┴─┐
q_1: ┤ Ry(θ[1]) ├┤ X ├
     └──────────┘└───┘


In [43]:
theta = [np.pi/4,0]
job = estimator.run([(psi0, H2, [theta])])
job_result = job.result() # It will block until the job finishes.
print("Result: ", job_result[0].data.evs)

Result:  [0.70710678]


In [35]:
print(H2)

SparsePauliOp(['IZ'],
              coeffs=[1.+0.j])
